In [ ]:
# Author: Arthur BARREAU
# Date: 2025-07-15
# R version: 4.3.3
# Description: This script takes a date range and subarea as input,and outputs
#              a text file containing the CDS API request parameters.

# ==============================================================================
# 0. PACKAGE & LIBRARY INSTALLATION ----
# ==============================================================================

system("conda install -y r-sf=1.0_20")

library(sf)
library(dplyr)
library(jsonlite)

# ==============================================================================
# 1. GLOBAL OPTIONS & CONFIG LOADING ----
# ==============================================================================
options(timeout = 3600)

# Load configuration from JSON
config <-  fromJSON("galaxy_inputs/galaxy_inputs.json")

list_year <- as.numeric(unlist(strsplit(json_data$year, ",")))
list_year <- seq(list_year[1],list_year[2])

formatted_years <- sprintf("\"%s\"", list_year)  # Ajoute des guillemets autour de chaque année
formatted_years <- paste(formatted_years, collapse = ", ")  # Fusionne les années avec une virgule et un espace



subarea <- json_data$subarea
area_list <- unlist(strsplit(subarea, ","))

download_ASDs <- function() {
  output_dir <- "asd"
  dir.create(output_dir, showWarnings = FALSE)
  
  urls_asd <- c(
    "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.shp",
    "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.shx",
    "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.dbf",
    "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.prj",
    "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.cst"
  )
  
  destfiles <- file.path(output_dir, c(
    "asd-shapefile-EPSG4326.shp",
    "asd-shapefile-EPSG4326.shx",
    "asd-shapefile-EPSG4326.dbf",
    "asd-shapefile-EPSG4326.prj",
    "asd-shapefile-EPSG4326.cst"
  ))
  
  mapply(download.file, urls_asd, destfiles, MoreArgs = list(mode = "wb"))
  st_read(file.path(output_dir, "asd-shapefile-EPSG4326.shp"))
}

ASDs <- download_ASDs()
subareas_selected <- filter(ASDs, GAR_Short_ %in% area_list)
bbox <- st_bbox(subareas_selected) 

#bbox pas toujours dans le bonne ordre 
xmin <- min(bbox["xmin"], bbox["xmax"])
xmax <- max(bbox["xmin"], bbox["xmax"])
ymin <- min(bbox["ymin"], bbox["ymax"])
ymax <- max(bbox["ymin"], bbox["ymax"])


# Créer le texte avec les paramètres souhaités
script_text <- paste(
  "import cdsapi\n\n",
  "c = cdsapi.Client()\n",
  "c.retrieve(\"reanalysis-era5-single-levels\",  \n",
  "{\n",
  "    \"product_type\": [\"ensemble_mean\"],\n",
  "    \"variable\": [\"2m_temperature\"],\n",
  "    \"year\": [",formatted_years, "],\n",
  "    \"month\": [\n",
  "        \"01\", \"02\", \"03\",\n",
  "        \"04\", \"05\", \"06\",\n",
  "        \"07\", \"08\", \"09\",\n",
  "        \"10\", \"11\", \"12\"\n",
  "    ],\n",
  "    \"day\": [\n",
  "        \"01\", \"02\", \"03\",\n",
  "        \"04\", \"05\", \"06\",\n",
  "        \"07\", \"08\", \"09\",\n",
  "        \"10\", \"11\", \"12\",\n",
  "        \"13\", \"14\", \"15\",\n",
  "        \"16\", \"17\", \"18\",\n",
  "        \"19\", \"20\", \"21\",\n",
  "        \"22\", \"23\", \"24\",\n",
  "        \"25\", \"26\", \"27\",\n",
  "        \"28\", \"29\", \"30\",\n",
  "        \"31\"\n",
  "    ],\n",
  "    \"time\": [\n",
  "        \"00:00\", \"03:00\", \"06:00\",\n",
  "        \"09:00\", \"12:00\", \"15:00\",\n",
  "        \"18:00\", \"21:00\"\n",
  "    ],\n",
  "    \"data_format\": \"grib\",\n",
  "    \"download_format\": \"unarchived\",\n",
  "    \"area\": [", ymax, ", ", xmin, ", ",ymin, ", ",  xmax, "]\n",  # Correction bbox
  "},\n",
  "'download.nc')\n"
)

# Spécifier le chemin du fichier de sortie
output_file <- "outputs/script.txt"

# Écrire le texte dans le fichier
writeLines(script_text, output_file)

Reading layer `asd-shapefile-EPSG4326' from data source `/data/jwd02f/pulsar_staging/79954935/working/jupyter/asd/asd-shapefile-EPSG4326.shp' using driver `ESRI Shapefile'
Simple feature collection with 19 features and 16 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -180 ymin: -85.41189 xmax: 180 ymax: -45
Geodetic CRS:  WGS 84
